In [1]:
import pandas as pd
df = pd.read_pickle('kbl_preprocessed.pkl')

In [5]:
from bkcharts import Histogram
from bkcharts import defaults, show, output_file
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [9]:
defaults.width = 600
defaults.height = 350

# input options
hist = Histogram(pd.DataFrame(df[df['sen_length']<3000]['sen_length']), title="df['sen_length']", bins=100)
show(hist)

In [2]:
df=df[df['sen_length']>25]
df=df.reset_index(drop=True)

In [4]:
df.head(3)

level_0                                           contents  \
0      HH  [KBO리그] 전체적 전력 미흡으로 가을야구 실패했지만...[오마이뉴스 김승훈 기자...   
1      HH  (서울=연합뉴스) 천병혁 기자 = 신생팀 케이티 위즈가 통렬한 홈런포를 쏘아 올리며...   
2      HH  한화 이글스 선수단. 2015.9.30 뉴스1   News1 신성룡 기자(서울=뉴스...   

             datetime                         title  \
0 2015-10-03 20:43:03    아쉬운 마리한화... 그러나 박수가 아깝지 않다   
1 2015-10-03 16:47:21      케이티, 신생팀 최다승 타이 한화 PS 좌절   
2 2015-10-03 16:50:42  [프로야구]화제의 중심 한화, 결국 포스트시즌 탈락   

                                                 url  label_num  \
0  http://sports.news.naver.com/kbaseball/news/re...          1   
1  http://sports.news.naver.com/kbaseball/news/re...          1   
2  http://sports.news.naver.com/kbaseball/news/re...          1   

                                           sentences  sen_length  con_length  
0    전체적 전력 미흡으로 가을야구 실패했지만. 한화 이글스가 2009년부터 2014...        3393        3562  
1   신생팀 케이티 위즈가 통렬한 홈런포를 쏘아 올리며 한화 이글스의 실낱같은 희망마저...         838         987  
2  한화 이글스 선수단. 2015.9. 숱한 화제를 뿌렸던 한화 이글스의 2015시즌이...        2273        2386

In [5]:
X = df.sentences
y = df.label_num
print(X.shape)
print(y.shape)

(196993,)
(196993,)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(147744,)
(49249,)
(147744,)
(49249,)


In [7]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]

CPU times: user 28 ms, sys: 8 ms, total: 36 ms
Wall time: 98 ms


In [8]:
stopwords_kbo=['조','동안','때','구단','만','월','일','메이저','타율','실점','등판','명','치','원','피','프로','이날','상대','타선','연승','연패','상황','연속','이후','후','루','뒤','등','년','팀','승','것','월','위','패','전','번','군','개','점','차','중','기사','제공','팬','루수','회','말','초','야구','리그']

### Summary:
    * vect.fit(train) learns the vocabulary of the training data
    * vect.transform(train) uses the fitted vocabulary to build a document term matrix from the training data
    * vect.transform(test) uses the fitted vocabulary to build a document term matrix from the testing data(and ignores tokens it hasn't seen before)

In [9]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger,stop_words=stopwords_kbo, ngram_range=(1,3), min_df=2, max_df=.5)
X_train_dtm = vect.fit_transform(X_train)

CPU times: user 9min 59s, sys: 7.29 s, total: 10min 6s
Wall time: 10min 7s


In [10]:
X_test_dtm = vect.transform(X_test)

In [11]:
X_train_dtm.shape

(147744, 3095372)

In [12]:
X_test_dtm.shape

(49249, 3095372)

## save vector & dtm

https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python
https://stackoverflow.com/questions/32764991/how-do-i-store-a-tfidfvectorizer-for-future-use-in-scikit-learn

In [14]:
import pickle

with open("kbo_tfidf_vect.pkl", 'wb') as handle:
    pickle.dump(vect, handle)

with open("kbo_dtm.pkl", 'wb') as handle:
    pickle.dump(X_test_dtm, handle)

In [15]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.01)

In [16]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 2.74 s, sys: 304 ms, total: 3.05 s
Wall time: 3.05 s


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [17]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [18]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.94284147901480231

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class, target_names=['두산', '한화', 'LG', '넥센', '기아', '삼성', 'NC', '롯데', 'SK', 'KT']))

             precision    recall  f1-score   support

         두산       0.94      0.95      0.95      7479
         한화       0.95      0.96      0.96      5976
         LG       0.94      0.94      0.94      5125
         넥센       0.94      0.94      0.94      5192
         기아       0.93      0.94      0.94      4857
         삼성       0.92      0.94      0.93      4540
         NC       0.95      0.93      0.94      4405
         롯데       0.95      0.95      0.95      4415
         SK       0.95      0.92      0.94      4345
         KT       0.94      0.93      0.94      2915

avg / total       0.94      0.94      0.94     49249



In [20]:
import numpy as np
counts = np.bincount(y_pred_class[(y_pred_class!=0) & (y_test==0)])
print(np.argmax(counts))

5


In [21]:
X_test[(y_pred_class==5) & (y_test==0)]

126113      24일 10개 구단 관전 포인트 KBO리그 10개 구단의 키플레이어와 이슈를 한...
115498    강정호 공로상 수상이 발표됐다.한국 프로야구 OB회인 사단법인 일구회(회장 이재환)...
110516      삼성과 두산에서 가장 많은 도루를 기록한 박해민(왼쪽)과 오재원.삼성 라이온즈 ...
116500    삼성 라이온즈가 운명의 8회에 또다시 역전극을 만들어내며 4연승을 내달렸다.삼성은 ...
134520    1일 오후 대구 삼성라이온즈파크에서 열리는  2017 프로야구  삼성 라이온즈와 두...
121690      삼성의 FA 영입은 성공적일까. 이원석을 둔 손익계산서를 보면 애매한 느낌을 지...
128866      두산 베어스의 양의지와 오재원이 24일 대구 삼성라이온즈 파크에서 열린 삼성과의...
120180      삼성이 전 두산 내야수 최영진(28)을 영입했다.속초상고와 한일장신대를 거쳐 2...
127365      삼성 라이온즈가 화끈한 화력을 뽐내면서 시즌 첫 승을 신고했다. 삼성은 2일 대...
112991     불미스러운 일로 팬들께 심려를 끼쳐드려 죄송합니다. 삼성 라이온즈와 두산 베어스의...
116699      삼성이 3연승을 내달렸다. 삼성 라이온즈는 18일 서울 잠실야구장에서 열린 20...
114611      한국 프로야구 OB회인 사단법인 일구회(회장 이재환)는  2015 넷마블 마구마...
127276      4월 셋째주 포지션별 베스트 11과 워스트 11 프로야구 통계기록실 (케이비리포...
112463     차우찬.  삼성은 4차전 선발로 알프레도 피가로를 내세웠다. 두산의 4차전 선발은...
127818     두산 정수빈, 오재원, 삼성 구자욱, NC 나성범, 넥센 서건창 선수의 아트토이 ...
128186    두산 베어스 장원준은 지난해까지 통산 97승을 올려 올시즌 초반 100승 클럽에 가...
119376      상수보다는 변수, 예측불허라는 말이 어울렸다. 올 

In [22]:
df[df.index==116500]

level_0                                           contents  \
116500      OB  삼성 라이온즈가 운명의 8회에 또다시 역전극을 만들어내며 4연승을 내달렸다.삼성은 ...   

                  datetime                       title  \
116500 2015-08-19 21:57:03  삼성, 두산에 6대4 승리, 4경기 연속 역전승   

                                                      url  label_num  \
116500  http://sports.news.naver.com/kbaseball/news/re...          0   

                                                sentences  sen_length  \
116500  삼성 라이온즈가 운명의 8회에 또다시 역전극을 만들어내며 4연승을 내달렸다.삼성은 ...         861   

        con_length  
116500        1059

In practice Multinomial NB can be better than Linear SVM in some situations and svm better than NB on other datasets. Apparently t's possible to combine the two approaches to get a very good baseline: projects:nbsvm - Sida I. Wang

You can also try non-linear SVMs but often the quadratic complexity of the SMO algorithm (for instance as implemented in libsvm) makes it not practical on datasets with more than 5000 documents. Instead on prefer to use liblinear than can only train linear SVM on large datasets.

https://www.quora.com/What-are-the-differences-similarities-between-SVM-Naive-Bayes-for-binary-text-classification-wrt-how-they-are-processing-the-features

http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC

In [23]:
from sklearn import svm
#clf = svm.SVC(decision_function_shape='ovo')
clfrSVM=svm.LinearSVC(C=1)

In [24]:
%time clfrSVM.fit(X_train_dtm, y_train)

CPU times: user 1min 8s, sys: 568 ms, total: 1min 8s
Wall time: 1min 8s


LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [25]:
y_pred_class_svm = clfrSVM.predict(X_test_dtm)

In [26]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class_svm)

0.96706532112327159

In [27]:
print(classification_report(y_test, y_pred_class_svm, target_names=['두산', '한화', 'LG', '넥센', '기아', '삼성', 'NC', '롯데', 'SK', 'KT']))

             precision    recall  f1-score   support

         두산       0.97      0.97      0.97      7479
         한화       0.97      0.98      0.98      5976
         LG       0.96      0.96      0.96      5125
         넥센       0.96      0.97      0.97      5192
         기아       0.96      0.96      0.96      4857
         삼성       0.96      0.96      0.96      4540
         NC       0.97      0.97      0.97      4405
         롯데       0.97      0.97      0.97      4415
         SK       0.97      0.97      0.97      4345
         KT       0.96      0.96      0.96      2915

avg / total       0.97      0.97      0.97     49249



https://stackoverflow.com/questions/40115043/no-space-left-on-device-error-while-fitting-sklearn-model

In [28]:
with open("kbo_svm_clf.pkl", 'wb') as handle:
    pickle.dump(clfrSVM, handle)

In [29]:
with open("kbo_nb_clf.pkl", 'wb') as handle:
    pickle.dump(nb, handle)